In [150]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 28 11:38:35 2023 

The MPC controller takes in the previous RTN state computed (not the measured 'x' variable)
"""

import math
import matplotlib.pyplot as plt
import numpy as np

from math import sqrt, sin, asin, cos, tan, atan2, pi
from numpy.linalg import norm, inv, pinv
from scipy.linalg import expm, sinm, cosm

from source import spacecraft
from aa273_control import *
import matplotlib.patches as patches


In [151]:
##############################################################################
##############################################################################
###                                                                        ###
###              AUXILIARY FUNCTIONS FOR GPS CONSTELLATION                 ###
###                                                                        ###
##############################################################################
##############################################################################
def get_hill_frame(r, v):
    h = np.cross(r, v)  # Angular momentum vector
    r_hat = r / np.linalg.norm(r)  # Local X-axis (R)
    h_hat = h / np.linalg.norm(h)  # Local Z-axis (N)
    y_hat = np.cross(h_hat, r_hat)  # Local Y-axis (T)
    return np.array([r_hat, y_hat, h_hat])

def rtn_to_eci(target_ECI, chaser_RTN):
    r_target = target_ECI[0:3]
    v_target = target_ECI[3:6]
    r_chaser = chaser_RTN[0:3]
    v_chaser = chaser_RTN[3:6]
    matrix_rtn2eci = get_hill_frame(r_target, v_target)
    nuDot = norm(np.cross(r_target, v_target)) / (norm(r_target)**2)
    omega = np.array([0.0, 0.0, nuDot])
    rho = np.transpose(matrix_rtn2eci) @ r_chaser
    rhoDot = np.transpose(matrix_rtn2eci) @ (v_chaser + np.cross(omega, r_chaser))
    
    chaser_relative_ECI = np.zeros(6)
    chaser_relative_ECI[0:3] = rho
    chaser_relative_ECI[3:6] = rhoDot

    chaser_ECI = chaser_relative_ECI + target_ECI
    return chaser_ECI # Chaser in ECI frame

def eci_to_rtn(target_ECI, chaser_ECI):
    r_target = target_ECI[0:3]
    v_target = target_ECI[3:6]

    chaser_relative_ECI = chaser_ECI - target_ECI
    rho = chaser_relative_ECI[0:3]
    rhoDot = chaser_relative_ECI[3:6]

    nuDot = norm(np.cross(r_target, v_target)) / (norm(r_target)**2)
    omega = np.array([0.0, 0.0, nuDot])
    matrix_rtn2eci = get_hill_frame(r_target, v_target)

    r_chaser = matrix_rtn2eci @ rho
    v_chaser = matrix_rtn2eci @ rhoDot - np.cross(omega, r_chaser)

    chaser_RTN = np.zeros(6)
    chaser_RTN[0:3] = r_chaser
    chaser_RTN[3:6] = v_chaser

    return chaser_RTN # Chaser in RTN frame



In [152]:

##############################################################################
##############################################################################
###                                                                        ###
###               SETTING UP OF THE ACTUAL FLIGHT SIMULATION               ###
###                                                                        ###
##############################################################################
##############################################################################
dt = 0.2
duration = 500
timesteps = math.floor(duration / dt)


P_cost = np.eye(6)*1e5 # terminal state cost matrix
Q_cost = np.diag([1e2, 1e2, 1e2, 1e-3, 1e-3, 1e-3]) # state cost matrix
R_cost = 1*np.eye(3) # control cost matrix

N = 15 # MPC step horizon
max_iters = 200 # maximum number of iterations for SCP  
eps = 1 # tolerance for SCP


#Define Spacecrafts
sc1_elements = [6918.14, 0.00001, 97.59760, 0.000000, -109.33800, -99.025]
sc1 = spacecraft.Spacecraft( elements = sc1_elements )

target_sc_elements = [6918.14, 0.00001, 97.59760, 0.000000, -109.33800,  -99]
sc_target = spacecraft.Spacecraft( elements = target_sc_elements )


sc1.forces['maneuvers'] = True
sc1.set_force_frame('ECI')


# Initialize the state history
chaser_ECI_history = np.zeros((timesteps, 6))
target_ECI_history = np.zeros((timesteps, 6))

chaser_RTN_history = np.zeros((timesteps, 6))

chaser_MPC_history = np.zeros((timesteps, N+1, 6))

thrust_history = np.zeros((timesteps, 3))
error_history = np.zeros(timesteps)

u_ECI = np.zeros(3)

In [153]:
##############################################################################
##############################################################################
###                                                                        ###
###                    ACTUAL FLIGHT SIMULATION BELOW                      ###
###                                                                        ###
##############################################################################
##############################################################################
print(timesteps)

# For loop, generate ground truth `xt` and noisy measurements `y`.
for k in range(timesteps):
    print('---Iteration: ', k)
    # Propagate the true states of all spacecraft using RK4 propagator
    sc1.set_thruster_acceleration(u_ECI/sc1.mass) # u_ECI/sc1.mass
    sc1.propagate_perturbed(dt, dt)
    
    sc_target.propagate_orbit(dt)
    
    # Record states
    target_eci_k = np.array([sc_target.px, sc_target.py, sc_target.pz, sc_target.vx, sc_target.vy, sc_target.vz])
    chaser_eci_k = np.array([sc1.px, sc1.py, sc1.pz, sc1.vx, sc1.vy, sc1.vz])
    chaser_ECI_history[k, :] = chaser_eci_k
    target_ECI_history[k, :] = target_eci_k
    
    chaser_RTN = eci_to_rtn(target_eci_k, chaser_eci_k)
    chaser_RTN_history[k, :] = chaser_RTN

    # Check error between target and chaser aka how far chaser is from origin in RTN frame
    error = np.linalg.norm(chaser_RTN[0:3])
    if (error < 3e-3):
        print('Target reached!')
        print('Error: ', error)
        error_history[k] = error
        break
    elif (error > 2e3):
        print('all hope is lost :(')
        print('Error: ', error)
        error_history[k] = error
        break
    else:
        print('Error: ', error)
        error_history[k] = error

    # u_ECI = np.zeros(3)

    # Use MPC to compute the optimal control input
    # MPC begins at previous state (takes in RTN) and attempts to reach origin in RTN frame
    x_mpc, u_mpc = do_MPC(dt, sc1.n, sc1.mass, chaser_RTN, np.zeros(6), N, Q_cost, R_cost, P_cost, max_iters, eps)
    chaser_MPC_history[k, :, :] = x_mpc[:, :]

    # Convert the MPC output to ECI
    eci_2_hill = get_hill_frame(target_eci_k[0:3], target_eci_k[3:6])
    hill_2_eci = np.transpose(eci_2_hill)
    u_ECI =  hill_2_eci @ u_mpc[1, :]
    # u_ECI = u_mpc[1, :]
    thrust_history[k, :] = u_ECI

    


2500
---Iteration:  0
Error:  3.018603281504709
Converged in 2 iterationss
---Iteration:  1
Error:  3.018594329899666
Converged in 2 iterationss
---Iteration:  2
Error:  3.0185674230043915
Converged in 2 iterationss
---Iteration:  3
Error:  3.0185225172102794
Converged in 2 iterationss
---Iteration:  4
Error:  3.018459584877346
Converged in 2 iterationss
---Iteration:  5
Error:  3.018378607049274
Converged in 2 iterationss
---Iteration:  6
Error:  3.0182795620042664
Converged in 2 iterationss
---Iteration:  7
Error:  3.0181624283816593
Converged in 2 iterationss
---Iteration:  8
Error:  3.0180271822544573
Converged in 2 iterationss
---Iteration:  9
Error:  3.0178737976287913
Converged in 2 iterationss
---Iteration:  10
Error:  3.017702250184165
Converged in 2 iterationss
---Iteration:  11
Error:  3.0175125066257826
Converged in 2 iterationss
---Iteration:  12
Error:  3.0173045317121083
Converged in 2 iterationss
---Iteration:  13
Error:  3.017078295261331
Converged in 2 iterationss
---

In [154]:
# print(u_mpc)
min_err = np.min(error_history[0:k+1])
print('Minimum error: ', min_err)
min_err_idx = np.argmin(error_history[0:k+1])
print('Minimum error index: ', min_err_idx)
improvement = (error_history[0] - min_err)
print('Improvement: ', improvement)

# print position at min error
print('Position at min error: ', chaser_RTN_history[min_err_idx, 0:3])

final_it = k+1

docked_traj = []
sc_target.mass += sc1.mass
for m in range(math.floor(6000/dt)-1):
    sc_target.propagate_orbit(dt)
    docked_eci_k = np.array([sc_target.px, sc_target.py, sc_target.pz, sc_target.vx, sc_target.vy, sc_target.vz])
    docked_traj.append(docked_eci_k)
docked_traj = np.array(docked_traj)
# print([np.linalg.norm(u_mpc[i], np.inf) for i in range(N-1)])

Minimum error:  0.0017323445381186688
Minimum error index:  475
Improvement:  3.0168709369665905
Position at min error:  [-3.00991602e-04 -1.70599580e-03 -1.69079587e-13]


In [168]:
# ##############################################################################
# ##############################################################################
# ###                                                                        ###
# ###                               PLOTTING                                 ###
# ###                                                                        ###
# ##############################################################################
# ##############################################################################
print(timesteps)
%matplotlib tk
# Plot the trajectory of the chaser and target in ECI frame
plt.figure()

plt.plot(docked_traj[1:, 0], docked_traj[1:, 1], 'g', label='Docked Trajectory')
plt.plot(chaser_ECI_history[1:k, 0], chaser_ECI_history[1:k, 1], 'b', label='Chaser Trajectory')
plt.plot(target_ECI_history[1:k, 0], target_ECI_history[1:k, 1], 'r', label='Target Trajectory')
plt.plot(chaser_ECI_history[1, 0], chaser_ECI_history[1, 1], '*', label='Chaser Initial', color = 'magenta')
plt.plot(chaser_ECI_history[k, 0], chaser_ECI_history[k, 1], '*', markersize=15,label='Chaser Final', color = 'orange')
plt.plot(target_ECI_history[1, 0], target_ECI_history[1, 1], '*', label='Target Initial', color = 'blue')
plt.plot(target_ECI_history[k, 0], target_ECI_history[k, 1], 'k*', label='Target Final', color = 'black')

plt.xlabel('X [km]')
plt.ylabel('Y [km]')
plt.title('Trajectory of Chaser and Target in ECI frame')
plt.legend()

# Plot the trajectory of the chaser in RTN frame
plt.figure()
# plt.axis('equal')
fig = plt.gcf()
ax = fig.gca()
plt.plot(chaser_RTN_history[1:k, 0], chaser_RTN_history[1:k, 1], 'bo', label='Chaser')
plt.plot(chaser_RTN_history[1, 0], chaser_RTN_history[1, 1], '*', label='Chaser Initial', markersize=15, color = 'magenta')
plt.plot(chaser_RTN_history[k, 0], chaser_RTN_history[k, 1], '*', label='Chaser Final', markersize=15, color = 'orange')

time_arr = np.arange(0, math.floor(k / dt), 20)
# for t in time_arr:
#     plt.plot(chaser_MPC_history[t, :5, 0], chaser_MPC_history[t, :5, 1], '--*', color='k')
plt.plot(0, 0, 'x', label='Target', color = 'black')
plt.plot(chaser_RTN_history[min_err_idx, 0], chaser_RTN_history[min_err_idx, 1], 'x', label='Minimum Error')
tolerance_rad = patches.Ellipse((0, 0), 3e-3, 3e-3, color='orange', fill = False)
ax.add_patch(tolerance_rad)
plt.xlabel('R [km]')
plt.ylabel('T [km]')
plt.title('Trajectory of Chaser in RTN frame')
plt.legend()

# Plot 3D plot
plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = chaser_RTN_history[1:, 2]
xline = chaser_RTN_history[1:, 0]
yline = chaser_RTN_history[1:, 1]
ax.plot3D(xline, yline, zline, 'blue', label='Chaser')
ax.plot3D(xline[0], yline[0], zline[0], '*', label='Chaser Initial')
ax.plot3D(xline[-1], yline[-1], zline[-1], '*', label='Chaser Final')
ax.plot3D(0, 0, 0, 'x', label='Target')
ax.set_xlabel('X [km]')
ax.set_ylabel('Y [km]')
ax.set_zlabel('Z [km]')

# Plot thrust history
plt.figure()
plt.plot(thrust_history[:k+20, 0], label='x')
plt.plot(thrust_history[:k+20, 1], label='y')
plt.plot(thrust_history[:k+20, 2], label='z')
plt.xlabel('Time [s]')
plt.ylabel('Thrust [N]')
plt.title('Thrust History')
plt.legend()

# Plot error history
plt.figure()
plt.plot(error_history[0:k+20])
plt.xlabel('Time [s]')
plt.ylabel('Error [km]')
plt.title('Error History (L2-norm distance from origin)')



2500


C:\Users\conni\AppData\Local\Temp\ipykernel_25752\1799474292.py:19: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k*" (-> color='k'). The keyword argument will take precedence.
  plt.plot(target_ECI_history[k, 0], target_ECI_history[k, 1], 'k*', label='Target Final', color = 'black')


Text(0.5, 1.0, 'Error History (L2-norm distance from origin)')

In [ ]:

# plt.close('all')


# # Convert km to m
# x_history  = x_history * 1000
# xtrue_history = xtrue_history * 1000
# target_true_history = target_true_history * 1000

# # Trajectory
# plt.figure()
# plt.title('Trajectory')
# q = 0
# plt.plot( xtrue_history[0, q:], xtrue_history[1, q:], 'b', label=' Chaser Truth')
# plt.plot( x_history[0, q:], x_history[1, q:], 'r', label=' Chaser Estimate')
# plt.plot(x_history[0, q], x_history[1, q], '*', label=' Chaser Start')
# plt.xlabel('X [km]')

# plt.plot(target_true_history[0,:], target_true_history[1,:], 'g', label=' Target Truth')
# plt.legend()

# plt.figure()


# # RTN Positions
# plt.title('RTN Positions')
# for t in range (29):
#     plt.plot(x_MPC_history[0, 0:4, t], x_MPC_history[1, 0:4, t], '--*', color='k')
# plt.plot(x_RTN_history[0,:], x_RTN_history[1,:], '-o', label=' Chaser Estimate')
# plt.plot(x_RTN_truth[0,:], x_RTN_truth[1,:], '-o', label=' Chaser Truth')
# plt.plot(x_RTN_history[0, q], x_RTN_history[1, q], '*', label=' Chaser Position')
# plt.plot(x_RTN_history[0, 0], x_RTN_history[1, 0], '*', label=' Chaser Start')
# plt.plot(0, 0, '*', label=' Target')

# plt.legend()

# plt.figure()
# plt.title('Control input')
# plt.plot(u_history[0,:], label='u_x')
# plt.plot(u_history[1,:], label='u_y')
# plt.plot(u_history[2,:], label='u_z')
# plt.xlim(0, 30)
# plt.ylabel('Force')
# plt.xlabel('Time steps (3000 sec)')
# plt.legend()

# #show plot
# plt.show()
